<a href="https://colab.research.google.com/github/Katmalu/hse22_hw2/blob/main/src/Katmalu_hse22_HW2_biopython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Скачиваем данные и устанавливаем программы

In [1]:
!wget https://raw.githubusercontent.com/Katmalu/hse22_hw2/main/Data/gms2.lst

--2022-10-22 17:36:34--  https://raw.githubusercontent.com/Katmalu/hse22_hw2/main/Data/gms2.lst
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221149 (216K) [text/plain]
Saving to: ‘gms2.lst.1’

gms2.lst.1          100%[===================>] 215.97K  --.-KB/s    in 0.02s   

2022-10-22 17:36:35 (11.0 MB/s) - ‘gms2.lst.1’ saved [221149/221149]



In [2]:
!wget https://raw.githubusercontent.com/Katmalu/hse22_hw2/main/Data/scaffold.fa

--2022-10-22 17:36:44--  https://raw.githubusercontent.com/Katmalu/hse22_hw2/main/Data/scaffold.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3967033 (3.8M) [text/plain]
Saving to: ‘scaffold.fa.1’

scaffold.fa.1       100%[===================>]   3.78M  --.-KB/s    in 0.05s   

2022-10-22 17:36:44 (79.9 MB/s) - ‘scaffold.fa.1’ saved [3967033/3967033]



In [3]:
!wget https://raw.githubusercontent.com/Katmalu/hse22_hw2/main/Data/proteins.fasta

--2022-10-22 17:36:51--  https://raw.githubusercontent.com/Katmalu/hse22_hw2/main/Data/proteins.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1404012 (1.3M) [text/plain]
Saving to: ‘proteins.fasta.1’

proteins.fasta.1    100%[===================>]   1.34M  --.-KB/s    in 0.03s   

2022-10-22 17:36:51 (43.3 MB/s) - ‘proteins.fasta.1’ saved [1404012/1404012]



In [4]:
!wget https://raw.githubusercontent.com/Katmalu/hse22_hw2/main/Data/scaffolds.hits_from_MIL_1.txt

--2022-10-22 17:36:55--  https://raw.githubusercontent.com/Katmalu/hse22_hw2/main/Data/scaffolds.hits_from_MIL_1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 869163 (849K) [text/plain]
Saving to: ‘scaffolds.hits_from_MIL_1.txt.1’

scaffolds.hits_from 100%[===================>] 848.79K  --.-KB/s    in 0.03s   

2022-10-22 17:36:56 (24.9 MB/s) - ‘scaffolds.hits_from_MIL_1.txt.1’ saved [869163/869163]



In [5]:
!wget https://raw.githubusercontent.com/Katmalu/hse22_hw2/main/Data/scaffolds.hits_from_SwissProt.txt

--2022-10-22 17:37:03--  https://raw.githubusercontent.com/Katmalu/hse22_hw2/main/Data/scaffolds.hits_from_SwissProt.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66190 (65K) [text/plain]
Saving to: ‘scaffolds.hits_from_SwissProt.txt.1’

scaffolds.hits_from 100%[===================>]  64.64K  --.-KB/s    in 0.006s  

2022-10-22 17:37:03 (10.5 MB/s) - ‘scaffolds.hits_from_SwissProt.txt.1’ saved [66190/66190]



## Скачиваем геном близкородственной бактерии T.oleivorans

Геном, последовательности генов (нт) и белков (протеом) для бактерии Thalassolituus oleivorans MIL-1
https://www.ncbi.nlm.nih.gov/nuccore/HF680312

In [6]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.


To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [7]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [8]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Аннотация генома (пример)

Biopython Tutorial: http://biopython.org/DIST/docs/tutorial/Tutorial.html

In [9]:
from Bio import SeqIO

records = dict()
for record in SeqIO.parse("scaffold.fa", "fasta"):
  record.annotations['molecule_type'] = 'DNA'
  records[record.id] = record

## Добавляем координаты и трансляции генов

In [10]:
from Bio.SeqFeature import SeqFeature, FeatureLocation

genes = dict()
for record in SeqIO.parse("proteins.fasta", "fasta"):
    id, sc_name, start, end, strend, *rest = record.description.split()
    if strend == '+':
      strend = 1
    else:
      strend = -1
    Feature = SeqFeature(FeatureLocation(int(start), int(end), strand = strend), type="CDS")
    Feature.qualifiers['locus_tag'] = [id]
    Feature.qualifiers['translation'] = [record.seq]
    records[sc_name].features.append(Feature)
    genes[record.name] = Feature

## Добавляем функции белков (из бактерии MIL-1)

In [11]:
import pandas as pd

data = pd.read_csv("scaffolds.hits_from_MIL_1.txt", sep='\t', names =['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore'])
data

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,1,lcl|HF680312.1_prot_CCU71653.1_1204,98.990,99,1,0,1,99,1,99,8.080000e-68,196.0
1,1,lcl|HF680312.1_prot_CCU72283.1_1834,97.980,99,2,0,1,99,1,99,2.290000e-67,194.0
2,1,lcl|HF680312.1_prot_CCU71933.1_1484,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
3,1,lcl|HF680312.1_prot_CCU71592.1_1143,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
4,1,lcl|HF680312.1_prot_CCU71865.1_1416,96.774,62,2,0,1,62,1,62,2.150000e-42,130.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10504,3613,lcl|HF680312.1_prot_1833,100.000,52,0,0,1,52,95,146,5.410000e-34,109.0
10505,3613,lcl|HF680312.1_prot_CCU71654.1_1205,100.000,52,0,0,1,52,95,146,2.250000e-33,110.0
10506,3613,lcl|HF680312.1_prot_CCU71593.1_1144,100.000,52,0,0,1,52,95,146,2.510000e-33,110.0
10507,3613,lcl|HF680312.1_prot_CCU71934.1_1485,100.000,52,0,0,1,52,95,146,4.490000e-33,110.0


In [12]:
hits = data.sort_values('bitscore').drop_duplicates('qseqid', keep='last')
hits = hits.sort_values('qseqid')
hits

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,1,lcl|HF680312.1_prot_CCU71653.1_1204,98.990,99,1,0,1,99,1,99,8.080000e-68,196.0
7,3,lcl|HF680312.1_prot_CCU71928.1_1479,100.000,173,0,0,1,173,1,173,1.660000e-130,359.0
8,5,lcl|HF680312.1_prot_CCU71924.1_1475,97.143,35,1,0,1,35,1,35,1.510000e-19,75.1
9,6,lcl|HF680312.1_prot_CCU71587.1_1138,98.770,244,3,0,1,244,38,281,2.170000e-171,470.0
11,7,lcl|HF680312.1_prot_CCU71586.1_1137,100.000,391,0,0,1,391,1,391,0.000000e+00,810.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10471,3607,lcl|HF680312.1_prot_CCU73035.1_2586,100.000,35,0,0,1,35,38,72,9.440000e-22,76.6
10476,3608,lcl|HF680312.1_prot_CCU70835.1_386,45.033,151,78,4,2,150,2735,2882,1.070000e-23,92.8
10485,3609,lcl|HF680312.1_prot_CCU71715.1_1266,100.000,63,0,0,1,63,160,222,6.030000e-39,130.0
10496,3612,lcl|HF680312.1_prot_CCU70866.1_417,100.000,48,0,0,1,48,428,475,2.550000e-29,103.0


In [13]:
mil_1_genome = SeqIO.read("/content/T_oleivorans_MIL_1.gbk", "genbank")

In [14]:
mil1genes = dict()
for  mil_f in mil_1_genome.features:
  if 'protein_id' not in mil_f.qualifiers:
    continue
    
  mil1genes[mil_f.qualifiers['protein_id'][0]] = mil_f.qualifiers['product'][0]

In [15]:
for _, hit in hits.iterrows():
  genes[str(hit['qseqid'])].qualifiers['product']= [mil1genes[hit['sseqid'].split('_')[2]]]

In [18]:
print(f"Генов, аннотированных с помощью сравнения с бактерией MIL-1: {hits.shape[0]}")

Генов, аннотированных с помощью сравнения с бактерией MIL-1: 3334


## Добавляем функции белков (из БД SwissProt)

Читаем информацию из файла scaffolds.hits_from_SwissProt.txt

In [16]:
!head scaffolds.hits_from_SwissProt.txt

169	sp|P55546|Y4LF_SINFN	42.553	141	80	1	1	141	1	140	1.18e-36	144
428	sp|Q06916|GUFA_MYXXA	43.802	242	136	0	71	312	13	254	2.33e-47	162
428	sp|Q3B4G1|ZUPT_CHLL3	34.016	244	148	6	72	304	17	258	6.82e-25	103
428	sp|Q8N1S5|S39AB_HUMAN	40.625	160	83	3	162	312	186	342	1.64e-23	101
428	sp|Q6P6S2|S39AB_RAT	40.000	160	84	3	162	312	179	335	2.03e-23	101
428	sp|Q2YDD4|S39AB_BOVIN	40.000	160	84	3	162	312	185	341	2.62e-23	101
428	sp|Q8NQK0|ZUPT_CORGL	35.968	253	144	7	67	306	10	257	1.83e-22	97.4
428	sp|Q8BWY7|S39AB_MOUSE	41.176	153	78	3	169	312	193	342	2.70e-22	98.6
428	sp|B3ECE6|ZUPT_CHLL2	34.000	250	146	10	71	306	16	260	5.36e-22	96.3
428	sp|Q28J44|S39AB_XENTR	39.623	159	84	3	163	312	181	336	1.08e-21	96.7


product = 'Similar to SwissProt protein Y1178_METJA (E-value = 2.95e-28)'

Получить функции белков SwissProt из файла: https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

In [17]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

--2022-10-22 17:39:05--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642093634 (612M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 612.35M  7.47MB/s    in 46s     

2022-10-22 17:39:51 (13.2 MB/s) - ‘uniprot_sprot.dat.gz’ saved [642093634/642093634]



In [19]:
!gzip -d uniprot_sprot.dat.gz

In [20]:
!head -n 50 uniprot_sprot.dat

ID   001R_FRG3G              Reviewed;         256 AA.
AC   Q6GZX4;
DT   28-JUN-2011, integrated into UniProtKB/Swiss-Prot.
DT   19-JUL-2004, sequence version 1.
DT   02-JUN-2021, entry version 42.
DE   RecName: Full=Putative transcription factor 001R;
GN   ORFNames=FV3-001R;
OS   Frog virus 3 (isolate Goorha) (FV-3).
OC   Viruses; Varidnaviria; Bamfordvirae; Nucleocytoviricota; Megaviricetes;
OC   Pimascovirales; Iridoviridae; Alphairidovirinae; Ranavirus.
OX   NCBI_TaxID=654924;
OH   NCBI_TaxID=30343; Dryophytes versicolor (chameleon treefrog).
OH   NCBI_TaxID=8404; Lithobates pipiens (Northern leopard frog) (Rana pipiens).
OH   NCBI_TaxID=45438; Lithobates sylvaticus (Wood frog) (Rana sylvatica).
OH   NCBI_TaxID=8316; Notophthalmus viridescens (Eastern newt) (Triturus viridescens).
RN   [1]
RP   NUCLEOTIDE SEQUENCE [LARGE SCALE GENOMIC DNA].
RX   PubMed=15165820; DOI=10.1016/j.virol.2004.02.019;
RA   Tan W.G., Barkman T.J., Gregory Chinchar V., Essani K.;
RT   "Comparative genomic a

In [21]:
!grep "^ID" uniprot_sprot.dat > uniprot_sprot_ID.dat

In [27]:
!grep "DE   RecName: Full=" uniprot_sprot.dat > uniprot_sprot_DE.dat

In [34]:
swgenes = dict()
for id, de in zip(open("uniprot_sprot_ID.dat"), open("uniprot_sprot_DE.dat")):
  ID = id.strip().split()[1]
  swgenes[ID] = de.split('=')[1][:-2]

In [29]:
data = pd.read_csv("scaffolds.hits_from_SwissProt.txt", sep='\t', names =['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore'])
data

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,169,sp|P55546|Y4LF_SINFN,42.553,141,80,1,1,141,1,140,1.180000e-36,144.0
1,428,sp|Q06916|GUFA_MYXXA,43.802,242,136,0,71,312,13,254,2.330000e-47,162.0
2,428,sp|Q3B4G1|ZUPT_CHLL3,34.016,244,148,6,72,304,17,258,6.820000e-25,103.0
3,428,sp|Q8N1S5|S39AB_HUMAN,40.625,160,83,3,162,312,186,342,1.640000e-23,101.0
4,428,sp|Q6P6S2|S39AB_RAT,40.000,160,84,3,162,312,179,335,2.030000e-23,101.0
...,...,...,...,...,...,...,...,...,...,...,...,...
928,3500,sp|P0A114|CLCD_PSEPU,29.439,214,136,6,97,299,19,228,4.730000e-15,76.3
929,3500,sp|P0A115|CLCD_PSEKB,29.439,214,136,6,97,299,19,228,4.730000e-15,76.3
930,3500,sp|P71505|DLHH_METEA,27.442,215,132,7,107,302,79,288,7.300000e-12,68.2
931,3500,sp|P27136|TFDE1_CUPPJ,24.171,211,148,3,98,299,20,227,2.800000e-11,65.5


In [31]:
swhits = data.sort_values('bitscore').drop_duplicates('qseqid', keep='last')
swhits = swhits.sort_values('qseqid')
swhits

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,169,sp|P55546|Y4LF_SINFN,42.553,141,80,1,1,141,1,140,1.180000e-36,144.0
1,428,sp|Q06916|GUFA_MYXXA,43.802,242,136,0,71,312,13,254,2.330000e-47,162.0
69,591,sp|P08956|T1RK_ECOLI,40.701,1199,642,23,8,1171,5,1169,0.000000e+00,837.0
74,597,sp|Q9V1E9|KTHY_PYRAB,39.073,151,76,5,3,138,4,153,3.120000e-15,73.9
78,600,sp|P06990|T1SB_ECOLX,35.794,447,271,5,4,436,19,463,4.330000e-71,238.0
85,602,sp|A9VQW6|PTUB_BACMK,32.402,179,96,8,48,224,50,205,6.880000e-12,67.8
86,967,sp|P39213|INSN_SHIDY,46.392,97,51,1,7,103,17,112,3.770000e-27,99.0
89,968,sp|P39315|QOR2_ECOLI,55.705,149,66,0,3,151,132,280,3.510000e-48,160.0
90,974,sp|P37626|YHII_ECOLI,55.460,348,144,1,23,359,8,355,4.060000e-121,356.0
148,976,sp|P0AGH2|YHHJ_SHIFL,65.684,373,128,0,4,376,2,374,1.480000e-170,483.0


In [36]:
for _, hit in swhits.iterrows():
  genes[str(hit['qseqid'])].qualifiers['product']= [swgenes[hit['sseqid'].split('|')[-1]]]

In [38]:
print(f"Генов, аннотированных с помощью SwissProt: {len(swhits)}")

Генов, аннотированных с помощью SwissProt: 53


In [39]:
SeqIO.write(records.values(), "GENOME.gbk", "genbank")

71